In [1]:
import typer
from typing import Optional, List
from phi.assistant import Assistant
from phi.storage.assistant.postgres import PgAssistantStorage
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.pgvector import PgVector, SearchType

import os
from dotenv import load_dotenv


In [2]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")



In [3]:
db_url = "postgresql+psycopg://ai:ai@pgvector:5432/ai"
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=PgVector(table_name="recipes", db_url=db_url, search_type=SearchType.hybrid),
)

In [4]:
# Load the knowledge base: Comment out after first run
# knowledge_base.load(recreate=True, upsert=True)

storage = PgAssistantStorage(table_name="pdf_assistant", db_url=db_url)

INFO     Dropping collection

INFO     Table 'ai.recipes' does not exist.

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO     Upserted batch of 14 documents.

INFO     Added 14 documents to knowledge base

In [5]:

def pdf_assistant(new: bool = False, user: str = "user"):
    run_id: Optional[str] = None

    if not new:
        existing_run_ids: List[str] = storage.get_all_run_ids(user)
        if len(existing_run_ids) > 0:
            run_id = existing_run_ids[0]

    assistant = Assistant(
        run_id=run_id,
        user_id=user,
        knowledge_base=knowledge_base,
        storage=storage,
        # Show tool calls in the response
        show_tool_calls=True,
        # Enable the assistant to search the knowledge base
        search_knowledge=True,
        # Enable the assistant to read the chat history
        read_chat_history=True,
    )
    if run_id is None:
        run_id = assistant.run_id
        print(f"Started Run: {run_id}\n")
    else:
        print(f"Continuing Run: {run_id}\n")

    assistant.cli_app(markdown=True)

In [8]:
pdf_assistant(new=False, user="user")

Continuing Run: 89e20e15-029c-4455-a77f-a07acfb3672a



 😎 User :

Output()

 😎 User :

Output()

KeyboardInterrupt: 